In [25]:
import pandas as pd
import numpy as np
import jieba
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.metrics import recall_score,accuracy_score,precision_score
import warnings
warnings.filterwarnings('ignore')

In [36]:
#数据加载
data = pd.read_csv('G:/git/python检查文章、新闻、字段的抄袭情况/sqlResult.csv',encoding = 'gb18030' )
data.head()

,source,content
0,证券时报网,证券时报网（www.stcn.com）06月23日讯\r\n 据上证报道，6月初以来，...
1,证券时报网,?\r\n 巨丰早评：市场将再次探底\r\n 【巨丰观点】\r\n 周四大盘冲高回落，...
2,中国新闻网,中新网6月19日电 据外媒报道，美国底特律一名男子1976年因为一根头发被定谋杀罪，监禁41...
3,中国证券报?中证网,曹先生：风格转换前的阵痛\r\n 今日早盘两地低开，之后一度震荡走高，领涨的仍然是上证...
4,国际在线,6月21日，MSCI在官网发布公告称，从明年6月起将中国A股纳入MSCI新兴市场指数和MSC...


In [37]:
#step2.数据预处理1.删除content、source为空的行
data = data.dropna(axis=0,subset=['content'])
data = data.dropna(axis = 0,subset = ['source'])
#print(data.isnull().sum())

In [38]:
#2.结巴分词+加载停用词
with open('G:/git/python检查文章、新闻、字段的抄袭情况/chinese_stopwords.txt',encoding='utf-8') as file:
    stopwords = [i[:-1] for i in file.readlines()]
def word_split(text):
    text = text.replace(' ','').replace('/n','')
    text2 = jieba.cut(text)
    result = ' '.join([t for t in text2 if t not in stopwords])
    return result
print(stopwords)
print(data.iloc[0].content)
print(word_split(data.iloc[0].content))
corpus = list(map(word_split,[str(i) for i in data.content]))

['，', '的', '。', '、', '在', '了', '是', '\u3000', '“', '”', '和', '年', '月', '：', '也', '）', '为', '（', '有', '%', '日', '将', '中', '-', '到', '与', '对', ':', '\xa0', '上', '都', '等', '不', '他', '》', '《', '就', '但', '我', '而', '这', '会', '并', '；', '被', '后', '人', '从', '还', '1', '3', '6', '以', '新', '说', '7', '2', '要', '5', '？', '更', '于', '个', '10', '大', '时', '4', '多', '/', '让', '其', ')', '(', '很', '及', '下', '', '能', '—', '或', '该', '她', '比', '8', '元', '12', '已', '向', '做', '来', '前', '由', '好', '.', '称', '给', '最', '11', '·', '据', '着', '又', '至', '9', '20', '！', '[', ']', '去', '可', '把', '则', '', '一', '地', '高', '吗', '30', '所', '分', '较', '内', '第', '里', '占', '过', '15', '曾', '"', '再', '人民日报', '新闻网', '它', '况', '而且', '而是', '而外', '而言', '而已', '尔后', '反过来', '反过来说', '反之', '非但', '非徒', '否则', '嘎', '嘎登', '该', '赶', '个', '各', '各个', '各位', '各种', '各自', '给', '根据', '跟', '故', '故此', '固然', '关于', '管', '归', '果然', '果真', '过', '哈', '哈哈', '呵', '和', '何', '何处', '何况', '何时', '嘿', '哼', '哼唷', '呼哧', '乎', '哗', '还是', '还有', '换句话说', '换言之', '或', '或是', '或

In [39]:
#step3.查看文本的特征，TFIDF
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
countVectorizer = CountVectorizer(encoding='gb18030',min_df=0.015)
tfidfTransformer = TfidfTransformer()
countVectorizer = countVectorizer.fit_transform(corpus)
tfidfTransformer = tfidfTransformer.fit_transform(countVectorizer)
print(tfidfTransformer)

  (0, 872)	0.2748530113256343
  (0, 751)	0.1599775402306061
  (0, 680)	0.275536443497438
  (0, 541)	0.24225528190125376
  (0, 483)	0.2765781627658592
  (0, 474)	0.201034310506804
  (0, 167)	0.26523990159520294
  (0, 145)	0.20082444910636033
  (0, 129)	0.495278556655558
  (0, 103)	0.1898018275595141
  (0, 87)	0.2681572143273556
  (0, 63)	0.26739648803453225
  (0, 62)	0.26926784288473765
  (0, 14)	0.19855021923226232
  (1, 878)	0.044777531795851845
  (1, 873)	0.04186966757610768
  (1, 865)	0.041079580613136546
  (1, 861)	0.12729721112473844
  (1, 859)	0.04617035623718872
  (1, 849)	0.27343142416141625
  (1, 844)	0.04030710844140916
  (1, 832)	0.04993485406473139
  (1, 818)	0.08290372138929573
  (1, 811)	0.03993774548205976
  (1, 806)	0.0804476074095681
  :	:
  (87051, 225)	0.05160074917487412
  (87051, 216)	0.12471484046406585
  (87051, 211)	0.05310357891749532
  (87051, 208)	0.09808041994954449
  (87051, 171)	0.054267720606178425
  (87051, 157)	0.05462116405416567
  (87051, 151)	0.05834

In [51]:
#step4标记是否为新华社文章,切分数据集
label = list(map(lambda i:1 if '新华社' in str(i) else 0,data.source))
#数据集切分
x_train,x_test,y_train,y_test = train_test_split(tfidfTransformer.toarray(),label,test_size=0.2)
#多项式朴素贝叶斯、伯努利贝叶斯分类准确率
model_m = MultinomialNB()
model_m.fit(x_train,y_train)
pred_m = model_m.predict(x_test)
print('MultinomialNB的准确率',accuracy_score(y_test,pred_m))
print('MultinomialNB的精确率',precision_score(y_test,pred_m))
print('MultinomialNB的召回率',recall_score(y_test,pred_m))

MultinomialNB的准确率 0.8878869680087301
MultinomialNB的精确率 0.9646047417556586
MultinomialNB的召回率 0.9097358586178501


In [54]:
#step5.预测文章风格是否和自己一致
prediction = model_m.predict(tfidfTransformer.toarray())
label = np.array(label)
compare = pd.DataFrame({'label':label , 'pred':prediction})
copy_index= compare[(compare['label'] == 0)&(compare['pred']==1)].index
print('可能为copy的新闻条数：',len(copy_index))
#实际为新华社的新闻
xinhuashe_news_index = compare[(compare['label'] == 1)].index

可能为copy的新闻条数： 2784


In [55]:
#step6 对tfidf.toarray()进行聚类降维
from sklearn.preprocessing import Normalizer
from sklearn.cluster import KMeans
normalizer = Normalizer()
scale_array = normalizer.fit_transform(tfidfTransformer.toarray())
kmeans = KMeans(n_clusters=25)
k_label = kmeans.fit_predict(scale_array)
#对分类创建序号
id_class = {index:class_ for index,class_ in enumerate(k_label)}

In [56]:
#step7创建新华社发布的字典
from collections import defaultdict
class_id = defaultdict(set)
for index,class_ in id_class.items():
    if index in xinhuashe_news_index.tolist():
        class_id[class_].add(index)
#查找相似的文章
from sklearn.metrics.pairwise import cosine_similarity
#只在新华社发布的文章中查找
def find_similar_text(cpindex,top=10):
    dist_dict = {i:cosine_similarity(tfidfTransformer[cpindex],tfidfTransformer[i]) for i in class_id[id_class[cpindex]]}
    return sorted(dist_dict.items(),key = lambda  x:x[1][0],reverse=True)[:top]


In [57]:
#step8.如查找某一篇是否为copy的,如果是抄袭的，查看可能抄袭的文章
cpindex = 2447
print('是否在新华社',cpindex in xinhuashe_news_index)
print('是否在copy_news',cpindex in copy_index)
similar_list = find_similar_text(cpindex)
print(similar_list)

print('怀疑抄袭：\n',data.iloc[cpindex].content)
#找一篇相似的原文
similar2 = similar_list[0][0]
print('相似的原文:\n',data.iloc[similar2].content)

是否在新华社 False
是否在copy_news False
[(12024, array([[0.50824924]])), (61536, array([[0.48992039]])), (74254, array([[0.46678522]])), (74313, array([[0.46678522]])), (42564, array([[0.4597049]])), (66150, array([[0.45058751]])), (50065, array([[0.45026396]])), (37008, array([[0.44543087]])), (82581, array([[0.4418654]])), (34693, array([[0.44147324]]))]
怀疑抄袭：
 　　原标题：斯瑞德、鼎义互联、精英在线等3家公司今日挂牌新三板
　　中国网财经6月6日讯 今日共有3家企业挂牌新三板，其中，协议转做市0家。截止今日，新三板市场总挂牌数达11263家，协议转让企业9703家，做市转让企业1560家；申报及待挂牌的企业共918家。
　　今日挂牌企业信息一览：
　　名称 主办券商 行业 斯瑞德
　　871488 第一创业证券股份有限公司 资本品 鼎义互联
　　871558 国融证券股份有限公司 软件与服务 精英在线
　　871575 安信证券股份有限公司 软件与服务
?

相似的原文:
 　　新华社北京１１月２５日电（记者王都鹏、刘慧）中国证监会２５日核准１４家企业的首发申请，这１４家企业的筹资总额预计不超过１１２亿元。\n　　１４家企业中，有７家企业拟在上海证券交易所上市，包括亚翔系统集成科技（苏州）股份有限公司、贵州省广播电视信息网络股份有限公司、上海元祖梦果子股份有限公司、浙江华正新材料股份有限公司、杭叉集团股份有限公司、日月重工股份有限公司、中原证券股份有限公司。有３家企业拟在深圳证券交易所中小板上市，包括比音勒芬服饰股份有限公司、深圳市同为数码科技股份有限公司、广州弘亚数控机械股份有限公司。拟在创业板上市的４家企业，为北京数字认证股份有限公司、英飞特电子（杭州）股份有限公司、西安晨曦航空科技股份有限公司、无锡贝斯特精机股份有限公司。\n　　证监会通报的信息显示，本次将有北京数字认证股份有限公司、西安晨曦航空科技股份有限公司等